Datasets used: 
- `adult (Adult)` :<br>The UCI Adult dataset contains 14 features, which can be divided into two categories: demographic and income-related.<br> The demographic features include:

    age: continuous.<br>
    fnlwgt : continuous, represents final weight, which is the number of units in the target population that the responding unit represents.<br>
    workclass: categorical, with values 'Private', 'Local-gov', 'Self-emp-not-inc', 'Federal-gov', 'State-gov', 'Self-emp-inc', 'Without-pay'.<br>
    education: categorical, with values '11th', 'HS-grad', 'Assoc-acdm', 'Some-college', '10th', ..., '9th', 'Doctorate', '12th', '1st-4th', 'Preschool'.<br>
    education-num: continuous with values 1 to 16, one number assigned to each label of education feature.<br>
    marital-status: categorical, with values 'Never-married', 'Married-civ-spouse', 'Widowed', 'Separated', 'Divorced', 'Married-spouse-absent', 'Married-AF-spouse'.<br>
    occupation: categorical, with values such as 'Machine-op-inspct', 'Farming-fishing', 'Protective-serv'.<br>
    relationship: categorical, with values 'Own-child', 'Husband', 'Not-in-family', 'Unmarried', 'Wife', 'Other-relative'.<br>
    race: categorical, with values such as white, black, and Asian.<br>
    sex: categorical, with values male and female.<br>
    
    The income-related features include:<br>

    hours-per-week: continuous.<br>
    native-country: categorical, with values such as United-States, Mexico, and Germany.<br>
    capital-gain: continuous, represent the amount of money an individual has gained from the sale of investments such as stocks, bonds, or real estate.<br>
    capital-loss: continuous, represent the amount of money an individual has lost from the sale of investments such as stocks, bonds, or real estate..<br>
    The target feature is the income, which is binary:<br>
    income: categorical, with values less than or equal to 50K and greater than 50K.<br>

Reference Links: https://archive.ics.uci.edu/ml/datasets/adult

In [1]:
import pandas as pd
import numpy as np

import os
import sys
module_path = os.path.abspath(os.path.join('..', '..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import warnings
warnings.filterwarnings('ignore')
import facts
from facts.clean import clean_dataset
from facts import valid_ifthens_with_coverage_correctness, rules2rulesbyif
from facts.models import customLogisticRegression
from facts.parameters import ParameterProxy
from facts.formatting import recourse_report_reverse, print_recourse_report, print_recourse_report_cumulative, print_recourse_report_KStest_cumulative
from facts.utils import load_rules_by_if
from facts.fairness_metrics_aggr import get_diff_table, get_comb_df, get_analysis_dfs,get_diff_real_diff_drop,get_other_ranks_divided,get_map_metric_to_max_rank

In [2]:
from pathlib import Path

random_seed = 131313 # for reproducibility
precomputed_rules_file = Path("..") / ".." / "Pre-computed Results" / "Adult" / "rulesAdult_race_atomic.data"
precomputed_rules_with_cumulative_file = Path("..") / ".." / "Pre-computed Results" / "Adult" / "rulesAdult_race_cumulative.data"

In [3]:
cost_budget = 10
top_count = 20
c_inf = 5

# Dataset

In [4]:
urlfile= "https://raw.githubusercontent.com/columbia/fairtest/master/data/adult/adult.csv"

X = pd.read_csv(urlfile)
df = clean_dataset(X, "adult")

age = [val.left for val in df.age.unique()]
age.sort()

df.head()

,age,Workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,"(34.0, 41.0]",State-gov,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,FullTime,United-States,0
1,"(41.0, 50.0]",Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,Married,White,Male,0,0,PartTime,United-States,0
2,"(34.0, 41.0]",Private,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,FullTime,United-States,0
3,"(50.0, 90.0]",Private,7,Married-civ-spouse,Handlers-cleaners,Married,Black,Male,0,0,FullTime,United-States,0
4,"(26.0, 34.0]",Private,13,Married-civ-spouse,Prof-specialty,Married,Black,Female,0,0,FullTime,Cuba,0


In [5]:
y = df['income']
X = df.drop('income', axis=1)

num_features = X._get_numeric_data().columns.to_list()
cate_features = X.select_dtypes(include=['object','category']).columns.to_list()
ord_features = ['hours-per-week']

# Rules

In [17]:
rules_with_atomic_correctness = load_rules_by_if(precomputed_rules_file)
rules_with_cumulative_correctness = load_rules_by_if(precomputed_rules_with_cumulative_file)

In [18]:
feature_weights = {"race" : 100 , "sex": 100, "marital-status": 5, "relationship":5, "age": 10, "occupation": 4, "Workclass": 2, "native-country": 4,
                    "hours-per-week":2, "capital-gain": 1, "capital-loss":1, "education-num": 3}
features_with_binary_cost = cate_features
features_with_proportional_cost = num_features


comparators = facts.feature_change_builder(
    X,
    num_cols=features_with_proportional_cost,
    cate_cols=features_with_binary_cost,
    ord_cols=ord_features,
    feature_weights=feature_weights,
    num_normalization=True,
    feats_to_normalize = ["capital-gain","capital-loss"]
)
params = ParameterProxy(featureChanges=comparators)

# Macro Viewpoint

## Equal Cost of Effectiveness (Macro)

In [19]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric="min-above-thr",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cor_threshold=0.7,
    filter_sequence = [
        "remove-contained",
        "remove-below-thr",        
        "remove-fair-rules",
        "keep-only-min-change"
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    metric_name = 'Equal Cost of Effectiveness (Macro) (threshold = 0.7)'
)

If age = (26.0, 34.0], marital-status =  Never-married, occupation =  Prof-specialty, relationship =  Not-in-family:
	Protected Subgroup ' Non-White', 1.15% covered
		Make age = (41.0, 50.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 80.00%.
		Make age = (34.0, 41.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 80.00%.
		Aggregate cost of the above recourses = 20.00
	Protected Subgroup ' White', 1.18% covered
		Make marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 75.47%.
		Aggregate cost of the above recourses = 10.00
	Bias against  Non-White due to Equal Cost of Effectiveness (Macro) (threshold = 0.7). Unfairness score = 10.
If age = (26.0, 34.0], education-num = 13, marital-status =  Never-married, relationship =  Not-in-family:
	Protected Subgroup ' Non-White', 1.78% covered
		Make age = (41.0, 50.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiv

## Equal Choice for Recourse

In [20]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric="num-above-thr",
    sort_strategy="generic-sorting-ignore-forall-subgroups-empty",
    top_count=top_count,
    cor_threshold=0.7,
    filter_sequence = [
        "remove-contained",
        "remove-below-thr",
        "remove-fair-rules"
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    metric_name = 'Equal Choice for Recourse(Macro) (threshold = 0.7)'
)

If age = (26.0, 34.0], marital-status =  Never-married, occupation =  Prof-specialty, relationship =  Not-in-family:
	Protected Subgroup ' Non-White', 1.15% covered
		Make age = (41.0, 50.0], marital-status =  Married-civ-spouse, occupation =  Exec-managerial, relationship =  Married with effectiveness 90.00%.
		Make age = (41.0, 50.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 80.00%.
		Make age = (34.0, 41.0], marital-status =  Married-civ-spouse, occupation =  Exec-managerial, relationship =  Married with effectiveness 80.00%.
		Make age = (34.0, 41.0], marital-status =  Married-civ-spouse, relationship =  Married with effectiveness 80.00%.
		Make age = (41.0, 50.0], marital-status =  Married-civ-spouse, occupation =  Sales, relationship =  Married with effectiveness 80.00%.
		Make age = (41.0, 50.0], marital-status =  Married-civ-spouse, occupation =  Tech-support, relationship =  Married with effectiveness 80.00%.
		Make age = (34.0, 41.0], m

# New Metrics

## Equal Conditional Mean Recourse

In [21]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="fairness-of-mean-recourse-conditional",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    filter_sequence = [
        "remove-contained",
        "remove-fair-rules"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    metric_name = 'Equal Conditional Mean Recourse'
)

If Workclass =  Private, capital-loss = 0, education-num = 10, hours-per-week = FullTime, marital-status =  Never-married, sex =  Male:
	Protected Subgroup ' Non-White', 2.35% covered
		Make marital-status =  Married-civ-spouse with effectiveness 2.44% and counterfactual cost = 1.0.
		Make hours-per-week = OverTime, marital-status =  Married-civ-spouse with effectiveness 2.44% and counterfactual cost = 2.0.
		Make hours-per-week = BrainDrain, marital-status =  Married-civ-spouse with effectiveness 2.44% and counterfactual cost = 3.0.
		Make education-num = 11, marital-status =  Married-civ-spouse with effectiveness 2.44% and counterfactual cost = 21.0.
		Make education-num = 11, hours-per-week = OverTime, marital-status =  Married-civ-spouse with effectiveness 2.44% and counterfactual cost = 22.0.
		Make education-num = 12, hours-per-week = OverTime, marital-status =  Married-civ-spouse with effectiveness 2.44% and counterfactual cost = 42.0.
		Aggregate cost of the above recourses = 1

## Equal Cost of Effectiveness (Micro)

In [22]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="min-above-corr",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cor_threshold = 0.7,
    filter_sequence = [
        "remove-contained",
        "remove-fair-rules",
        "keep-cheap-rules-above-thr-cor",
        
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    metric_name = 'Equal Cost of Effectiveness (Micro) (threshold = 0.7)'
)

If education-num = 9, marital-status =  Married-civ-spouse, occupation =  Transport-moving, sex =  Male:
	Protected Subgroup ' Non-White', 1.78% covered
		Make occupation =  Craft-repair with effectiveness 0.00% and counterfactual cost = 1.0.
		Make occupation =  Sales with effectiveness 6.45% and counterfactual cost = 1.0.
		Make occupation =  Exec-managerial with effectiveness 29.03% and counterfactual cost = 1.0.
		Make education-num = 10, occupation =  Craft-repair with effectiveness 29.03% and counterfactual cost = 21.0.
		Make education-num = 10, occupation =  Sales with effectiveness 29.03% and counterfactual cost = 21.0.
		Make education-num = 10, occupation =  Prof-specialty with effectiveness 29.03% and counterfactual cost = 21.0.
		Make education-num = 10, occupation =  Exec-managerial with effectiveness 41.94% and counterfactual cost = 21.0.
		Make education-num = 11, occupation =  Exec-managerial with effectiveness 70.97% and counterfactual cost = 41.0.
		Aggregate cost of

## Equal Effectiveness

In [23]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="total-correctness",
    sort_strategy="generic-sorting-ignore-forall-subgroups-empty",
    top_count=top_count,
    filter_sequence = [
        "remove-contained",
        "remove-fair-rules",
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    correctness_metric=True,
    metric_name = 'Equal Effectiveness'
)

If age = (34.0, 41.0], capital-loss = 0, native-country =  United-States, occupation =  Machine-op-inspct:
	Protected Subgroup ' Non-White', 1.38% covered
		Make occupation =  Exec-managerial with effectiveness 0.00% and counterfactual cost = 1.0.
		Make occupation =  Prof-specialty with effectiveness 0.00% and counterfactual cost = 1.0.
		Make occupation =  Sales with effectiveness 0.00% and counterfactual cost = 1.0.
		Make occupation =  Craft-repair with effectiveness 0.00% and counterfactual cost = 1.0.
		Make occupation =  Adm-clerical with effectiveness 0.00% and counterfactual cost = 1.0.
		Make occupation =  Tech-support with effectiveness 0.00% and counterfactual cost = 1.0.
		Make age = (41.0, 50.0], occupation =  Exec-managerial with effectiveness 0.00% and counterfactual cost = 2.0.
		Make age = (41.0, 50.0], occupation =  Prof-specialty with effectiveness 0.00% and counterfactual cost = 2.0.
		Make age = (50.0, 90.0], occupation =  Exec-managerial with effectiveness 0.00% 

## Fair Effectiveness-Cost Trade-Off

In [26]:
# taken from the other notebooks
affected_pop_sizes = {' Non-White': 1727, ' White': 9027}


top_rules,unfairness = facts.select_rules_subset_KStest(
    rules_with_cumulative_correctness,
    affected_pop_sizes,
    top_count=top_count
)

print_recourse_report_KStest_cumulative(
    top_rules,
    population_sizes=affected_pop_sizes,
    unfairness = unfairness,
    show_then_costs=True,
    metric_name = 'Fair Effectiveness-Cost Trade-Off'
    # show_cumulative_plots=True
)
    # show_cumulative_plots=True


If capital-gain = 0, native-country =  United-States:
	Protected Subgroup ' Non-White', 76.82% covered out of 1727
		Make capital-gain = 3103 with effectiveness 9.19% and counterfactual cost = 0.03.
		Make capital-gain = 4386 with effectiveness 12.99% and counterfactual cost = 0.04.
		Make capital-gain = 5178 with effectiveness 17.77% and counterfactual cost = 0.05.
		Make capital-gain = 7298 with effectiveness 27.71% and counterfactual cost = 0.07.
		Make capital-gain = 7688 with effectiveness 29.87% and counterfactual cost = 0.08.
		Make capital-gain = 15024 with effectiveness 74.46% and counterfactual cost = 0.15.
		Make capital-gain = 99999 with effectiveness 100.00% and counterfactual cost = 1.0.
	Protected Subgroup ' White', 89.37% covered out of 9027
		Make capital-gain = 3103 with effectiveness 20.20% and counterfactual cost = 0.03.
		Make capital-gain = 4386 with effectiveness 28.05% and counterfactual cost = 0.04.
		Make capital-gain = 5178 with effectiveness 32.66% and count

## Equal Effectiveness within Budget (Micro)

In [27]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="max-upto-cost",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cost_threshold = cost_budget,
    filter_sequence = [
        "remove-contained",
        "remove-fair-rules",
        "remove-above-thr-cost"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    correctness_metric = True,
    metric_name = 'Equal Effectiveness within Budget (Micro)'
)

If age = (34.0, 41.0], capital-loss = 0, native-country =  United-States, occupation =  Machine-op-inspct:
	Protected Subgroup ' Non-White', 1.38% covered
		Make occupation =  Exec-managerial with effectiveness 0.00% and counterfactual cost = 1.0.
		Make occupation =  Prof-specialty with effectiveness 0.00% and counterfactual cost = 1.0.
		Make occupation =  Sales with effectiveness 0.00% and counterfactual cost = 1.0.
		Make occupation =  Craft-repair with effectiveness 0.00% and counterfactual cost = 1.0.
		Make occupation =  Adm-clerical with effectiveness 0.00% and counterfactual cost = 1.0.
		Make occupation =  Tech-support with effectiveness 0.00% and counterfactual cost = 1.0.
		Make age = (41.0, 50.0], occupation =  Exec-managerial with effectiveness 0.00% and counterfactual cost = 2.0.
		Make age = (41.0, 50.0], occupation =  Prof-specialty with effectiveness 0.00% and counterfactual cost = 2.0.
		Make age = (50.0, 90.0], occupation =  Exec-managerial with effectiveness 0.00% 

## Get ranking of subgroups based on metrics

In [28]:
from facts.fairness_metrics_aggr import make_table, auto_budget_calculation

In [29]:
rules_with_both = {}
for ifc, all_thens in rules_with_cumulative_correctness.items():
    new_all_thens = {}
    for sg, (cov, thens_cum) in all_thens.items():
        thens_atom = rules_with_atomic_correctness[ifc][sg][1]
        thens_atom_dict = {then: atom_cor for then, atom_cor in thens_atom}
        new_all_thens[sg] = (cov, [(then, thens_atom_dict[then], cumcor) for then, cumcor, _cost in thens_cum])
    rules_with_both[ifc] = new_all_thens

In [30]:
budgets = auto_budget_calculation(rules_with_cumulative_correctness, cor_thres=0.5, percentiles=[0.3, 0.6, 0.9])

df = make_table(
    rules_with_both,
    sensitive_attribute_vals=[" Non-White", " White"],
    effectiveness_thresholds=[0.3, 0.7],
    cost_budgets=budgets,
    params=params
)
df.drop(['weighted-average', 'mean-cost-cinf'], axis=1, inplace=True)

In [31]:
#table of differences between Male and Female for each metric
diff = get_diff_table(df, sensitive_attribute_vals=[" White", " Non-White"], with_abs = True)
diff = diff.set_index('subgroup')
ranked = diff.drop(columns=[('Fair Effectiveness-Cost Trade-Off','bias')]).mask(diff == 0).rank(ascending = False,axis=0,method='dense')
ranked = ranked.replace(np.nan,"Fair")

In [32]:
rev_bias_metrics = ['Equal Effectiveness', 'Equal Effectiveness within Budget']
comb_df = get_comb_df(df, ranked, diff, rev_bias_metrics, sensitive_attribute_vals=[" White", " Non-White"])
diff_real_val, _, = get_diff_real_diff_drop(df, diff, sensitive_attribute_vals=[" White", " Non-White"])
analysis_df,rank_analysis_df = get_analysis_dfs(comb_df,diff_real_val, rev_bias_metrics, ranked,percentage=0.1, sensitive_attribute_vals=["White", "Non-White"])


In [33]:
comb_df.head()

(Equal Cost of Effectiveness(Macro), 0.3)  \
                                                                                       rank   
subgroup                                                                                      
capital-loss = 0                                                                       Fair   
capital-gain = 0                                                                       32.0   
capital-gain = 0, capital-loss = 0                                                     32.0   
capital-loss = 0, native-country =  United-States                                      Fair   
Workclass =  Private                                                                   Fair   

                                                                        \
                                                    score bias against   
subgroup                                                                 
capital-loss = 0                                   0.0000         Fair   
capital-gain = 0                                   0.0251    Non-White   
capital-gain = 0, capital-loss = 0                 0.0251    Non-White   
capital-loss = 0, native-country =  United-States  0.0000         Fair   
Workclass =  Private                               0.0000         Fair   

                                                  (Equal Cost of Effectiveness(Macro), 0.7)  \
                                                                                       rank   
subgroup                                                                                      
capital-loss = 0                                                                       Fair   
capital-gain = 0                                                                       Fair   
capital-gain = 0, capital-loss = 0                                                     Fair   
capital-loss = 0, native-country =  United-States                                      Fair   
Workclass =  Private                                                                   Fair   

                                                                      \
                                                  score bias against   
subgroup                                                               
capital-loss = 0                                    0.0         Fair   
capital-gain = 0                                    0.0         Fair   
capital-gain = 0, capital-loss = 0                  0.0         Fair   
capital-loss = 0, native-country =  United-States   0.0         Fair   
Workclass =  Private                                0.0         Fair   

                                                  (Equal Choice for Recourse, 0.3)  \
                                                                              rank   
subgroup                                                                             
capital-loss = 0                                                              Fair   
capital-gain = 0                                                               9.0   
capital-gain = 0, capital-loss = 0                                             9.0   
capital-loss = 0, native-country =  United-States                             Fair   
Workclass =  Private                                                          Fair   

                                                                      \
                                                  score bias against   
subgroup                                                               
capital-loss = 0                                    0.0         Fair   
capital-gain = 0                                    2.0    Non-White   
capital-gain = 0, capital-loss = 0                  2.0    Non-White   
capital-loss = 0, native-country =  United-States   0.0         Fair   
Workclass =  Private                                0.0         Fair   

                                                  (Equal Choice for Recourse, 0.7)  \
      

In [34]:
analysis_df

,Rank = 1 Count,White bias against Count,Non-White bias against Count,Top 10% White bias against Count,Top 10% Non-White bias against Count
"(Equal Cost of Effectiveness(Macro), 0.3)",1731,148,2802,0,295
"(Equal Cost of Effectiveness(Macro), 0.7)",325,58,526,7,51
"(Equal Choice for Recourse, 0.3)",1,215,3718,2,391
"(Equal Choice for Recourse, 0.7)",2,73,628,10,60
Equal Effectiveness,1,2512,11884,6,1433
"(Equal Effectiveness within Budget, 1.150241502415024)",1,54,189,0,24
"(Equal Effectiveness within Budget, 10.072980729807298)",1,2227,10316,3,1251
"(Equal Effectiveness within Budget, 21.0)",1,2490,11744,0,1423
"(Equal Cost of Effectiveness(Micro), 0.3)",1720,149,2794,0,294
"(Equal Cost of Effectiveness(Micro), 0.7)",325,58,522,7,51


In [35]:
max_ranks = get_map_metric_to_max_rank(ranked)
rank_divided = get_other_ranks_divided(rank_analysis_df, max_ranks)
rank_divided

,"(Equal Cost of Effectiveness(Macro), 0.3)","(Equal Cost of Effectiveness(Macro), 0.7)","(Equal Choice for Recourse, 0.3)","(Equal Choice for Recourse, 0.7)",Equal Effectiveness,"(Equal Effectiveness within Budget, 1.150241502415024)","(Equal Effectiveness within Budget, 10.072980729807298)","(Equal Effectiveness within Budget, 21.0)","(Equal Cost of Effectiveness(Micro), 0.3)","(Equal Cost of Effectiveness(Micro), 0.7)",Equal(Conditional) Mean Recourse,"(Fair Effectiveness-Cost Trade-Off, value)"
"(Equal Cost of Effectiveness(Macro), 0.3)",-,1.0,0.845,1.0,0.162,0.996,0.283,0.177,0.026,1.0,0.448,0.194
"(Equal Cost of Effectiveness(Macro), 0.7)",0.7,-,0.9,0.829,0.147,0.973,0.315,0.169,0.698,0.05,0.421,0.12
"(Equal Choice for Recourse, 0.3)",0.419,1.0,-,1.0,1.0,1.0,0.03,1.0,0.419,1.0,0.782,0.073
"(Equal Choice for Recourse, 0.7)",1.0,0.095,0.909,-,0.644,1.0,0.003,0.328,1.0,0.1,0.041,0.011
Equal Effectiveness,0.023,1.0,0.909,1.0,-,1.0,0.01,0.0,0.023,1.0,0.0,0.0
"(Equal Effectiveness within Budget, 1.150241502415024)",1.0,0.048,1.0,0.857,0.069,-,0.047,0.07,1.0,0.05,1.0,0.102
"(Equal Effectiveness within Budget, 10.072980729807298)",0.395,0.048,0.818,0.571,0.001,1.0,-,0.001,0.395,0.05,0.611,0.002
"(Equal Effectiveness within Budget, 21.0)",0.023,1.0,0.909,1.0,0.0,1.0,0.01,-,0.023,1.0,0.0,0.0
"(Equal Cost of Effectiveness(Micro), 0.3)",0.023,1.0,0.845,1.0,0.162,0.996,0.284,0.177,-,1.0,0.449,0.195
"(Equal Cost of Effectiveness(Micro), 0.7)",0.7,0.048,0.9,0.829,0.147,0.973,0.315,0.169,0.698,-,0.421,0.12
